<a href="https://colab.research.google.com/github/C4rohan/multivariate-weather-data-clustering/blob/main/Mock_netCDF_data_clustering_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install cftime
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import netCDF4
import netCDF4 as nc
import pandas as pd
import numpy as np
import xarray as xr
import datetime
import datetime as dt
from netCDF4 import date2num,num2date
import cftime
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import utils
import pandas as pd
import numpy as np
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates

%matplotlib inline

#Read Data#

In [ ]:
path2 = ('mock.nc')
data1 = xr.open_dataset(path2, decode_times=False) #To view the date as integers of 0, 1, 2,....
data5 = xr.open_dataset(path2) # To view time in datetime format
data1
#data5['time']

<xarray.Dataset>
Dimensions:  (time: 20, lat: 10, lon: 10)
Coordinates:
  * time     (time) int64 -9223372036854775806 ... -9223372036854775806
  * lat      (lat) float32 27.0 28.0 29.0 30.0 31.0 32.0 33.0 34.0 35.0 36.0
  * lon      (lon) float32 77.0 78.0 79.0 80.0 81.0 82.0 83.0 84.0 85.0 86.0
Data variables:
    sst      (time, lat, lon) float32 ...
    t2m      (time, lat, lon) float32 ...

<function xarray.core.dataarray.DataArray.shape>

In [ ]:

time=data5['time']
t2m=data5['t2m']
lat=data5['lat']
lon=data5['lon']

arr = np.array(lat)
np.size(arr)

10

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
sst = data5.sst
sst

In [ ]:
t2m = data5.t2m
t2m

#Caculate RMSE between 2 days of SST

In [ ]:
diff = sst[0] - sst[1]
Sq_diff = (diff**2)
Sq_diff


In [ ]:
Sq_diff_sum = sum(map(sum, Sq_diff.to_numpy()))
Sqrt_diff_sum = np.sqrt(Sq_diff_sum)
print("RMSE between SST[0] and SST[1]:", Sqrt_diff_sum)

RMSE between SST[0] and SST[1]: 0.0


In [ ]:
def rmse_2d(day1, day2):
  diff = day1 - day2
  Sq_diff = (diff**2)
  Sq_diff_sum = sum(map(sum, Sq_diff.to_numpy()))
  return np.sqrt(Sq_diff_sum)

##Intra Cluster RMSE

In [ ]:
mean_sst_1 = np.mean(sst[0:10], axis=0)
mean_sst_1

In [ ]:
mean_t2m_1 = np.mean(t2m[0:10], axis=0)
mean_t2m_1

In [ ]:
intra_rmse_value = 0.0
for i in range(len(sst[0:10])):
  intra_rmse_value += rmse_2d(sst[i], mean_sst_1)
print(i)
print("RMSE among SST[0:10]:", intra_rmse_value/(i+1))


In [ ]:
intra_rmse_value = 0.0
for i in range(len(sst[0:10])):
  intra_rmse_value += rmse_2d(sst[i], mean_sst_1)

for j in range(len(t2m[0:10])):
  intra_rmse_value += rmse_2d(t2m[j], mean_t2m_1)
intra_rmse_cluster_1 = intra_rmse_value/(i+1+j+1)
print("RMSE among cluster 1 (day1-day10):", intra_rmse_cluster_1)

RMSE among cluster 1 (day1-day10): 0.0


In [ ]:
intra_rmse_value = 0.0
mean_sst_2 = np.mean(sst[10:20], axis=0)
#
mean_t2m_2 = np.mean(t2m[10:20], axis=0)
for i in range(len(sst[10:20])):
  intra_rmse_value += rmse_2d(sst[i+10], mean_sst_2)

for j in range(len(t2m[10:20])):
  intra_rmse_value += rmse_2d(t2m[j+10], mean_t2m_2)
print("RMSE among cluster 2 (day11-day20):", intra_rmse_value/20)

##Inter Cluster RMSE

In [ ]:
inter_rmse_value_sst = rmse_2d(mean_sst_1, mean_sst_2)
print("RMSE between clusters 1 and 2 for SST:", inter_rmse_value_sst)
inter_rmse_value_t2m = rmse_2d(mean_t2m_1, mean_t2m_2)
print("RMSE between clusters 1 and 2 for T2M:", inter_rmse_value_t2m)
intra_rmse_value = (inter_rmse_value_sst + inter_rmse_value_t2m)/2
print("RMSE among clusters 1 and 2:", intra_rmse_value)